# Running ground-state upper-bound solver
In this example, we illustrate how to create an upper-bound solver from a cluster expansion and how to obtain a ground-state upper-bound solution.

In [1]:
import copy

import numpy as np
import matplotlib.pyplot as plt
from monty.serialization import loadfn, dumpfn
from pymatgen.analysis.ewald import EwaldSummation
from pymatgen.core import Structure

from smol.cofe import ClusterSubspace, ClusterExpansion
from smol.cofe.extern import EwaldTerm
from smol.cofe.space.domain import get_allowed_species

from smol.moca import Ensemble

from smol.capp.generate.groundstate.solver import GroundStateSolver

### 0) Create a Cluster Subspace based on the disordered structure with an Ewald term

In [2]:
lmof = loadfn('data/lmof_prim.json') 

In [3]:
subspace = ClusterSubspace.from_cutoffs(structure=lmof, 
                                        cutoffs={2:4.0, 3:3.0})

subspace.add_external_term(EwaldTerm(eta=None)) # Add the external Ewald Term

In [4]:
print("Species:", get_allowed_species(lmof))

Species: [[Species Li+, Species Mn2+, Vacancy vacA0+], [Species Li+, Species Mn2+, Species Mn3+, Species Mn4+, Vacancy vacA0+], [Species Li+, Species Mn2+, Vacancy vacA0+], [Species O2-, Species F-]]


In [5]:
print("Number of correlation functions (wo. Ewald):", subspace.num_corr_functions)

Number of correlation functions (wo. Ewald): 168


In [6]:
print("Number of orbits:", subspace.num_orbits)

Number of orbits: 30


### 1) Create random ECIs and ClusterExpansion

In [7]:
coefs = np.random.random(size=subspace.num_corr_functions+1)
coefs[0] = -10
coefs[-1] = 0.1

In [8]:
ce = ClusterExpansion(subspace, coefs)

### 2) Create a semi-grand canonical ensemble
By default, the Ensemble object uses ClusterDecompositionProcessor, which would greatly reduce the amount of many-body terms in pseudo-Boolean function. Switching to ClusterExpansionProcessor is allowed but not recommended.

The upper-bound supercell size is defined within the Ensemble. Here, we use a supercell containing 2 primitive units. A semigrand-canonical ensemble is defined as an Ensemble object with chemical potentials specified.

In [9]:
chempots = {"Li+": 0.0, "Mn2+": 0.0, "Mn3+": 0.0, "Mn4+": 0.0, "Vacancy":0.0, "O2-":0.0, "F-": 0.0}
grand_ens = Ensemble.from_cluster_expansion(ce, np.diag([2, 1, 1]),
                                            chemical_potentials=chempots)

### 3) Create a solver instance from the ensemble
Charge-balance constraints are included by default. If any other constraint is needed, refer to the documentation of GroundStateSolver.

The default solver is "SCIP". For other solvers supported by cvxpy, see: https://www.cvxpy.org/tutorial/advanced/index.html#setting-solver-options

In [10]:
# Use 1e-6 as a cutoff to cluster terms. Any term with coefficient lower than 1e-6 will not be included into the optimization.
grand_solver = GroundStateSolver(grand_ens, term_coefficients_cutoff=1e-6)
print("Number of variables:", grand_solver._canonicals.variables.size)
print("Number of auxiliary variables:", grand_solver._canonicals.auxiliary_variables.size)
print("Number of constraints:", len(grand_solver._canonicals.constraints))

Number of variables: 26
Number of auxiliary variables: 2997
Number of constraints: 11646


### 4) Solve the problem in semi-grand canonical ensemble

In [11]:
_, energy = grand_solver.solve()
structure = grand_solver.ground_state_structure

print("Ground-state energy, un-normalized(eV):", energy)
print("Ground-state structure:", structure)

Ground-state energy, un-normalized(eV): -26.155526605938583
Ground-state structure: Full Formula (Mn1 O2)
Reduced Formula: MnO2
abc   :   5.939700   2.969850   2.969850
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (3)
  #  SP       a    b    c
---  ----  ----  ---  ---
  0  Mn4+  0.5   0    0
  1  O2-   0.25  0.5  0.5
  2  O2-   0.75  0.5  0.5


### 5) Create and solve a canonical ensemble problem
Our solver also supports solving the ground-state in canonical ensembles. In doing so, one only needs to create a canonical ensemble (i.e., an ensemble with no chemical potentials provided). When using a canonical ensemble, note that either a fixed composition or an intial occupancy to determine the composition to fix must be provided as an argument to initialize the solver.

In [12]:
# Creating the ensemble.
canonical_ens = Ensemble.from_cluster_expansion(ce, np.diag([2, 1, 1]))
print("Sublattices:", canonical_ens.sublattices)
print("Bits:", [s.species for s in canonical_ens.sublattices])

Sublattices: [Sublattice(site_space=Li+0.3 Mn2+0.3 vacA0+0.4, sites=array([0, 1, 4, 5]), active_sites=array([0, 1, 4, 5]), encoding=array([0, 1, 2])), Sublattice(site_space=Mn4+0.25 Mn3+0.25 Mn2+0.25 Li+0.2 vacA0+0.05, sites=array([2, 3]), active_sites=array([2, 3]), encoding=array([0, 1, 2, 3, 4])), Sublattice(site_space=O2-0.5 F-0.5 , sites=array([6, 7]), active_sites=array([6, 7]), encoding=array([0, 1]))]
Bits: [(Species Li+, Species Mn2+, Vacancy vacA0+), (Species Li+, Species Mn2+, Species Mn3+, Species Mn4+, Vacancy vacA0+), (Species O2-, Species F-)]


In [13]:
# Fix to LiMnO2.
canonical_solver = GroundStateSolver(canonical_ens, term_coefficients_cutoff=1e-6, initial_occupancy=[0, 2, 2, 4 ,2 ,2 ,0, 0])
print("Number of variables:", canonical_solver._canonicals.variables.size)
print("Number of auxiliary variables:", canonical_solver._canonicals.auxiliary_variables.size)
print("Number of constraints:", len(canonical_solver._canonicals.constraints))

Number of variables: 26
Number of auxiliary variables: 2997
Number of constraints: 11655


In [14]:
_, energy = canonical_solver.solve()
structure = canonical_solver.ground_state_structure

print("Ground-state energy, un-normalized(eV):", energy)
print("Ground-state structure:", structure)

Ground-state energy, un-normalized(eV): -21.678626755538694
Ground-state structure: Full Formula (Li1 Mn1 O2)
Reduced Formula: LiMnO2
abc   :   5.939700   2.969850   2.969850
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (4)
  #  SP        a     b     c
---  ----  -----  ----  ----
  0  Li+   0.625  0.25  0.25
  1  Mn3+  0      0     0
  2  O2-   0.25   0.5   0.5
  3  O2-   0.75   0.5   0.5
